In [ ]:
from keras.models import *
from keras.layers import *
from keras.optimizers import RMSprop,SGD
from keras.preprocessing import image
from keras.callbacks import TensorBoard,History,EarlyStopping,CSVLogger
from keras.applications.inception_v3 import InceptionV3
from sklearn.utils import shuffle
import numpy as np
import pandas as pd
import h5py
import os

In [ ]:
train_path = './train'
validation_path = './validation'
test_path = './test'
assert(os.path.exists(train_path))
assert(os.path.exists(validation_path))
assert(os.path.exists(test_path))

nb_train_samples = 18047
nb_val_samples = 4377
image_size = (299,299)
batch_size = 128
class_num = 10

In [ ]:
base_model = InceptionV3(weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
# x = Dense(1024, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

# 构建我们需要训练的完整模型
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
train_datagen = image.ImageDataGenerator(rescale=1./255,
                                         shear_range=0.2,
                                         rotation_range=10,
# #                                          height_shift_range=0.1,
#                                          width_shift_range=0.1,
                                         channel_shift_range=5,
#                                          preprocessing_function=preprocess_input,
                                         horizontal_flip=True)

validation_datagen = image.ImageDataGenerator(rescale=1./255)
#                                              preprocessing_function=preprocess_input)


train_generator = train_datagen.flow_from_directory(
                                    train_path,
                                    target_size=image_size,
                                    batch_size=batch_size,
                                    class_mode='categorical',
                                    shuffle=True)

validation_generator = validation_datagen.flow_from_directory(
                                    validation_path,
                                    target_size=image_size,
                                    batch_size=batch_size,                                    
                                    class_mode='categorical',
                                    shuffle=False)


In [ ]:
for layer in base_model.layers:
    layer.trainable = False

# 编译模型（一定要在锁层以后操作）
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

resNet50_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=10,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/inceptionV3_fine_tune'),
                                   CSVLogger('inceptionV3'),
                                   EarlyStopping(patience=3)])

In [ ]:
# 让我们看看每一层的名字和层号，看看我们应该锁多少层呢：
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# 我们选择训练最上面的两个 Inception block
# 也就是说锁住前面249层，然后放开之后的层。
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

In [ ]:
model.compile(loss='categorical_crossentropy',
                    optimizer=optimizers.SGD(lr=0.0001, momentum=0.9),#,decay=1e-6,nesterov=True),
                    metrics=['accuracy'])  

model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=50,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/inceptionV3_fine_tune'),
                                   CSVLogger('inceptionV3_fine_tune')]) #,